In [1]:
from huggingface_hub import notebook_login

notebook_login(new_session=True, write_permission=True)

In [2]:
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer

WHERE = "rag_ucl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpt = "xaviergillard/rag_ucl"
tokenizer = AutoTokenizer.from_pretrained(checkpt)
model = AutoModelForMaskedLM.from_pretrained(checkpt, torch_dtype=torch.float32)

In [3]:
import pandas as pd

### PREPARE DATASET ####
dataset = pd.read_csv("corpus_pdf.csv").dropna()

input_ids = []
attention_mask = []
special_mask = []
for row in dataset.itertuples():
    text = row.text
    if text:
        tokenized = tokenizer(
            row.text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256,
            pad_to_multiple_of=256,
            return_overflowing_tokens=True,
            return_special_tokens_mask=True,
            stride=50,
        )
        input_ids.append(tokenized["input_ids"])
        attention_mask.append(tokenized["attention_mask"])
        special_mask.append(tokenized["special_tokens_mask"])


input_ids = np.concatenate(input_ids)
attention_mask = np.concatenate(attention_mask)
special_mask = np.concatenate(special_mask)

dst_train = pd.DataFrame(
    {"input_ids": list(input_ids), "attention_mask": list(attention_mask), "special_tokens_mask": list(special_mask)},
)
dst_train.to_parquet("./tokenized_rag.parquet", compression="gzip")
dst_train = Dataset.from_pandas(dst_train)
dst_train.save_to_disk("saved_to_disk")

Saving the dataset (0/1 shards):   0%|          | 0/27423 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

dst_train = load_from_disk("saved_to_disk")
dst_train

Dataset({
    features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 27423
})

In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
training_args = TrainingArguments(
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    save_total_limit=2,
    overwrite_output_dir=True,
    output_dir=WHERE,
    push_to_hub=True,
    hub_model_id="xaviergillard/rag_ucl",  # "xaviergillard/rag_ucl__based_on_xlmroberta",
    hub_strategy="checkpoint",
    # faster training
    max_grad_norm=8.0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dst_train,
    data_collator=collator,
    tokenizer=tokenizer,
)

trainer.train()

  0%|          | 0/10284 [00:00<?, ?it/s]

{'loss': 0.5249, 'grad_norm': 6.4557785987854, 'learning_rate': 4.756903928432517e-05, 'epoch': 0.15}
{'loss': 0.5047, 'grad_norm': 5.638826370239258, 'learning_rate': 4.513807856865033e-05, 'epoch': 0.29}
{'loss': 0.503, 'grad_norm': 5.803164958953857, 'learning_rate': 4.2707117852975496e-05, 'epoch': 0.44}
{'loss': 0.4843, 'grad_norm': 5.101504325866699, 'learning_rate': 4.027615713730066e-05, 'epoch': 0.58}
{'loss': 0.4745, 'grad_norm': 4.502727508544922, 'learning_rate': 3.7845196421625825e-05, 'epoch': 0.73}
{'loss': 0.472, 'grad_norm': 6.616970539093018, 'learning_rate': 3.541423570595099e-05, 'epoch': 0.88}
{'loss': 0.4777, 'grad_norm': 4.662632942199707, 'learning_rate': 3.298327499027616e-05, 'epoch': 1.02}
{'loss': 0.4613, 'grad_norm': 4.002581596374512, 'learning_rate': 3.0552314274601326e-05, 'epoch': 1.17}
{'loss': 0.4629, 'grad_norm': 5.761693477630615, 'learning_rate': 2.8121353558926487e-05, 'epoch': 1.31}
{'loss': 0.4898, 'grad_norm': 5.485171318054199, 'learning_rate'

TrainOutput(global_step=10284, training_loss=0.5830961232610085, metrics={'train_runtime': 3129.6915, 'train_samples_per_second': 26.287, 'train_steps_per_second': 3.286, 'total_flos': 1.0827276285247488e+16, 'train_loss': 0.5830961232610085, 'epoch': 3.0})

In [12]:
# let's make another round of it.
trainer.train()

  0%|          | 0/8570 [00:00<?, ?it/s]

{'loss': 3.1497, 'grad_norm': 7.78125, 'learning_rate': 4.70828471411902e-05, 'epoch': 0.29}
{'loss': 3.135, 'grad_norm': 7.65625, 'learning_rate': 4.41656942823804e-05, 'epoch': 0.58}
{'loss': 3.0942, 'grad_norm': 7.65625, 'learning_rate': 4.12485414235706e-05, 'epoch': 0.88}
{'loss': 3.0851, 'grad_norm': 7.34375, 'learning_rate': 3.8331388564760794e-05, 'epoch': 1.17}
{'loss': 3.0661, 'grad_norm': 8.125, 'learning_rate': 3.541423570595099e-05, 'epoch': 1.46}
{'loss': 3.0699, 'grad_norm': 7.59375, 'learning_rate': 3.249708284714119e-05, 'epoch': 1.75}
{'loss': 3.0466, 'grad_norm': 8.25, 'learning_rate': 2.957992998833139e-05, 'epoch': 2.04}
{'loss': 3.0444, 'grad_norm': 6.9375, 'learning_rate': 2.666277712952159e-05, 'epoch': 2.33}
{'loss': 3.057, 'grad_norm': 8.3125, 'learning_rate': 2.3745624270711785e-05, 'epoch': 2.63}
{'loss': 3.0562, 'grad_norm': 7.5, 'learning_rate': 2.0828471411901985e-05, 'epoch': 2.92}
{'loss': 3.0397, 'grad_norm': 7.8125, 'learning_rate': 1.791131855309218e

TrainOutput(global_step=8570, training_loss=3.065519253208868, metrics={'train_runtime': 2418.4897, 'train_samples_per_second': 56.694, 'train_steps_per_second': 3.544, 'total_flos': 1.804546047541248e+16, 'train_loss': 3.065519253208868, 'epoch': 5.0})

In [ ]:
# I'm still not happy with it ... (maybe that loading the model in f32 would be better)
trainer.train()